# 04. Inference and Data Loading with API

In this notebook we will try calling the API for inference and data loading. This notebook can be used as a reference how your application can use this solution to get the item recommendation and to load new item into the vector DB.

Refer to the diagram below for the flow of Notebook 04.

![Flow diagram](assets/notebook-04-flow.jpg)

### 1. Set up

In [1]:
!pip install aws_requests_auth -q
!pip install websocket-client -q

In [2]:
import requests, urllib, json
from aws_requests_auth.aws_auth import AWSRequestsAuth
import boto3
from helper.presigned_url import AWSAPIWebSocketPresignedURL

In [3]:
deployment_output = json.load(open("./deployment-output.json","r"))
api_url= deployment_output["RecommenderStack"]["apiurl"]
ws_api_endpoint = deployment_output["RecommenderStack"]["wsapiendpoint"]
ws_api_stage = deployment_output["RecommenderStack"]["wsapistage"]

### 2. Call the recommendation inference REST API

Let's use the REST API to get a recommendation of a place to travel given a traveller bio as input. This is an example only. Here is a part of the flow diagram above which depicts this process.

![Getting recommendation with REST API](assets/notebook-04-flow-step-2.jpg)

Create a new input depicted by "Input A" in the diagram above

In [4]:
new_input="""Female, 30 years old, married
I really want to celebrate my wedding anniversary with Husband. 
Somewhere where we can have a picnic with our own bento with wide view of sky and city."""

In [12]:
session = boto3.Session()
credentials = session.get_credentials()

payload = { 
    "text": new_input,
    "num_items": 1, # Optional, defaulting to the recommendation parameter you set in notebook 02 and uploaded in notebook 03
    "num_types": 1 # Optional, defaulting to the recommendation parameter you set in notebook 02 and uploaded in notebook 03
}

hostname = urllib.parse.urlparse(api_url).hostname 

auth = AWSRequestsAuth(aws_access_key=credentials.access_key,
                       aws_secret_access_key=credentials.secret_key,
                       aws_token=credentials.token,
                       aws_host=hostname,
                       aws_region=session.region_name,
                       aws_service='execute-api')

response = requests.post(api_url, auth=auth, json=payload, timeout=45)
print(response)
recommended_items = response.json()
recommended_items

<Response [200]>


{'items': [{'id': 2,
   'distance': 13.254677930739156,
   'description': "\nMarina Barrage\nCountry: Singapore\nThis is actually a dam built on the mouth of Marina Channel to supply fresh water to Singapore. It also functions as a flood control and as an attraction point for families.\nIt's roof has grass with path for people to walk up from ground. Families can fly kites on that roof or just simply have a wide view of the Singapore city's buildings.\nIt is also situated in the midst of cycling path that connects the East Coast Park cycling path and the city cycling path.\n"}]}

### 3. Call the recommendation inference WebSocket API

Since API Gateway REST API has a timeout of 29 seconds and sometimes the LLM takes more time to answer, we also deployed an API Gateway WebSocket API. Let's try to invoke it with the below code. This code (and the helper in separate file) is just an example to call the websocket endpoint using python client. The implementation can be different for other programming languages. Also, the example below also uses AWS SigV4 to authenticate with IAM since the WebSocket API is protected with IAM authentication.

Let's use the WebSocket API to get a recommendation of a place to travel given a traveller bio as input. This is an example only. Here is a part of the flow diagram above which depicts this process.

![Getting recommendation with WebSocket](assets/notebook-04-flow-step-3.jpg)

In [10]:
from websocket import create_connection

session = boto3.Session()
credentials = session.get_credentials()

path = f"/{ws_api_stage}"
host = urllib.parse.urlparse(ws_api_endpoint).hostname 
access_key = credentials.access_key
secret_key=credentials.secret_key
session_token=credentials.token
region = session.region_name

ws_url_generator = AWSAPIWebSocketPresignedURL(access_key, secret_key, session_token, ws_api_endpoint, path, host,  region)
request_url = ws_url_generator.get_request_url()

payload ={
    "action":"inference", 
    "text":new_input,
    "num_items": 1,  # Optional, defaulting to the recommendation parameter you set in notebook 02 and uploaded in notebook 03
    "num_types": 1 # Optional, defaulting to the recommendation parameter you set in notebook 02 and uploaded in notebook 03
}

ws = create_connection(request_url)
print("Sending data")
ws.send(json.dumps(payload))
print("Sent")
print("Receiving...")
result =  ws.recv()
ws.close()

json.loads(result)

Sending data
Sent
Receiving...


{'items': [{'id': 2,
   'distance': 13.254677930739156,
   'description': "\nMarina Barrage\nCountry: Singapore\nThis is actually a dam built on the mouth of Marina Channel to supply fresh water to Singapore. It also functions as a flood control and as an attraction point for families.\nIt's roof has grass with path for people to walk up from ground. Families can fly kites on that roof or just simply have a wide view of the Singapore city's buildings.\nIt is also situated in the midst of cycling path that connects the East Coast Park cycling path and the city cycling path.\n"}]}

### 4. Call the data insert REST API

Not only getting recommendation, this solution also allows you to add more data items into the database.


Let's use the REST API to insert a new place to travel in the database. We will then create a new traveller bio and test whether the recommender system can return our newly inserted data using the recommendation inference REST API. The use of travel recommendation context is an example only. Here is a part of the flow diagram above which depicts this process.

![Insert data and verify with REST API](assets/notebook-04-flow-step-4.jpg)

Create a new item depicted by "Item B" in the diagram above

In [11]:
new_item="""Place: Botanic Gardens
This wide area offers plenty of flowers for Singaporean and tourists to do sightseeing. People can go picnic there too.
It has a national orchid garden inside it. There is a dome with controlled temperature, offering a variety of orchid flowers. If you are lucky you might catch a monitor lizard too.
The park has multiple ponds, with swan to be seen in at least one of them.
"""

In [13]:
session = boto3.Session()
credentials = session.get_credentials()

payload = { "text": new_item }

hostname = urllib.parse.urlparse(api_url).hostname 

auth = AWSRequestsAuth(aws_access_key=credentials.access_key,
                       aws_secret_access_key=credentials.secret_key,
                       aws_token=credentials.token,
                       aws_host=hostname,
                       aws_region=session.region_name,
                       aws_service='execute-api')

response = requests.put(api_url, auth=auth, json=payload, timeout=45)
print(response)

<Response [200]>


Create a new input depicted by "Input B" in the diagram above

In [14]:
new_input="""Gender: Female
Request: I wanna see some flowers, especially orchid."""

Verify if the new data is already inserted and whether a specific input can trigger the recommender system to return recommend the newly inserted data.

In [15]:
session = boto3.Session()
credentials = session.get_credentials()

payload = { "text": new_input }

hostname = urllib.parse.urlparse(api_url).hostname 

auth = AWSRequestsAuth(aws_access_key=credentials.access_key,
                       aws_secret_access_key=credentials.secret_key,
                       aws_token=credentials.token,
                       aws_host=hostname,
                       aws_region=session.region_name,
                       aws_service='execute-api')

response = requests.post(api_url, auth=auth, json=payload, timeout=45)
recommended_items = response.json()
recommended_items

{'items': [{'id': 4,
   'distance': 10.578008983341084,
   'description': 'Place: Botanic Gardens\nThis wide area offers plenty of flowers for Singaporean and tourists to do sightseeing. People can go picnic there too.\nIt has a national orchid garden inside it. There is a dome with controlled temperature, offering a variety of orchid flowers. If you are lucky you might catch a monitor lizard too.\nThe park has multiple ponds, with swan to be seen in at least one of them.\n'}]}

### 5. Call data insert WebSocket API

We can also use WebSocket for inserting data


Let's use the WebSocket API to insert a new place to travel in the database. We will then create a new traveller bio and test whether the recommender system can return our newly inserted data using the recommendation inference WebSocket API. The use of travel recommendation context is an example only. Here is a part of the flow diagram above which depicts this process.

![Insert data and verify with WebSocket API](assets/notebook-04-flow-step-5.jpg)

Insert a new item, depicted by "Item C" in the diagram above

In [16]:
new_item="""Place: Mount Faber Cable Car
Description: This cable car spans from the peak of Mount Faber park to the Sentosa island. It passes through Harborfrount in the middle.
The view includes a sea landscape, the city view of Singapore, and the Sentosa island with its attractions.
You can use this cable car to have a complete roundtrip or to go to the Sentosa island.
"""

In [17]:
session = boto3.Session()
credentials = session.get_credentials()

path = f"/{ws_api_stage}"
host = urllib.parse.urlparse(ws_api_endpoint).hostname 
access_key = credentials.access_key
secret_key=credentials.secret_key
session_token=credentials.token
region = session.region_name

ws_url_generator = AWSAPIWebSocketPresignedURL(access_key, secret_key, session_token, ws_api_endpoint, path, host,  region)
request_url = ws_url_generator.get_request_url()

payload ={
    "action":"insertdata", 
    "text":new_item,
}

ws = create_connection(request_url)
print("Sending data")
ws.send(json.dumps(payload))
print("Sent")
print("Receiving...")
result =  ws.recv()
ws.close()

result

Sending data
Sent
Receiving...


'Data has been loaded'

Insert a new input, depicted by "Input C" in the diagram above

In [18]:
new_input="""In this trip to Singapore, would it be possible to have a journey to the Sentosa with view from the sky?"""

Verify if the new data is already inserted and whether a specific input can trigger the recommender system to return recommend the newly inserted data.

In [19]:
session = boto3.Session()
credentials = session.get_credentials()

path = f"/{ws_api_stage}"
host = urllib.parse.urlparse(ws_api_endpoint).hostname 
access_key = credentials.access_key
secret_key=credentials.secret_key
session_token=credentials.token
region = session.region_name

ws_url_generator = AWSAPIWebSocketPresignedURL(access_key, secret_key, session_token, ws_api_endpoint, path, host,  region)
request_url = ws_url_generator.get_request_url()

payload = {
    "action":"inference", 
    "text":new_input
}
    
ws = create_connection(request_url)
print("Sending data")
ws.send(json.dumps(payload))
print("Sent")
print("Receiving...")
result =  ws.recv()
print(result)
ws.close()

Sending data
Sent
Receiving...
{"items": [{"id": 5, "distance": 7.829937940689803, "description": "Place: Mount Faber Cable Car\nDescription: This cable car spans from the peak of Mount Faber park to the Sentosa island. It passes through Harborfrount in the middle.\nThe view includes a sea landscape, the city view of Singapore, and the Sentosa island with its attractions.\nYou can use this cable car to have a complete roundtrip or to go to the Sentosa island.\n"}]}
